# Probando por separado

In [1]:
from cmp.evaluation import evaluate_reverse_parse

from engine.Lexer.hulk_lexer import HulkLexer 
from engine.language.tokens_type import hulk_tokens
from engine.language.grammar import G
from engine.Parser.hulk_parser import HulkParser

hulk_lexer = HulkLexer(hulk_tokens, G.EOF)
hulk_parser = HulkParser()

In [2]:
from tests.test_cases import test_cases

tokens = []
right_parses = []
operations = []
asts = []

for input in test_cases:
    token = hulk_lexer(input)
    tokens.append(token)

    right_parse, operation = hulk_parser(token)
    right_parses.append(right_parse)
    operations.append(operation)

    ast = evaluate_reverse_parse(right_parse, operation, token)
    asts.append(ast)

    print(token)
    print(right_parse)
    print(operation)
    print(ast)
    print("="*100)


[TokenType.LET: let, TokenType.IDX: number, TokenType.EQUAL: =, TokenType.NUM: 42, TokenType.IN: in, TokenType.LET: let, TokenType.IDX: text, TokenType.EQUAL: =, TokenType.STRING: "The meaning of life is", TokenType.IN: in, TokenType.IDX: print, TokenType.OPAR: (, TokenType.IDX: text, TokenType.CONCAT: @, TokenType.IDX: number, TokenType.CPAR: ), TokenType.SEMI: ;, TokenType.EOF: $]
[<atomic_exp> -> number, <var_exp> -> <atomic_exp>, <not_exp> -> <var_exp>, <type_exp> -> <not_exp>, <pow_exp> -> <type_exp>, <factor> -> <pow_exp>, <term> -> <factor>, <arithmetic_exp> -> <term>, <concat_exp> -> <arithmetic_exp>, <is_as_exp> -> <concat_exp>, <comp_exp> -> <is_as_exp>, <boolean_exp> -> <comp_exp>, <assignment_exp> -> <boolean_exp>, <special_exp> -> <assignment_exp>, <expr> -> <special_exp>, <assignment> -> id = <expr>, <assignment_list> -> <assignment>, <atomic_exp> -> string, <var_exp> -> <atomic_exp>, <not_exp> -> <var_exp>, <type_exp> -> <not_exp>, <pow_exp> -> <type_exp>, <factor> -> <p

In [6]:

from engine.semantic.type_builder_visitor import TypeBuilder
# from engine.semantic.type_checker_visitor import TypeChecker
from engine.semantic.type_collector_visitor import TypeCollector
from engine.semantic.type_inferer_visitor import TypeInferer
from engine.semantic.variable_collector_visitor import VarCollector


def semantic_analysis_pipeline(ast, debug=False):
    # if debug:
    #     formatter = Formatter()
    #     formatted_ast = formatter.visit(ast)
    #     print('===================== AST =====================')
    #     print(formatted_ast)
    if debug:
        print('============== COLLECTING TYPES ===============')
    errors = []
    collector = TypeCollector(errors)
    collector.visit(ast)
    context = collector.context
    if debug:
        print('Errors: [')
        for error in errors:
            print('\t', error)
        print(']')
        print('Context:')
        print(context)
        print('=============== BUILDING TYPES ================')
    builder = TypeBuilder(context, errors)
    builder.visit(ast)
    if debug:
        print('Errors: [')
        for error in errors:
            print('\t', error)
        print(']')
        print('Context:')
        print(context)
        print('=============== CHECKING TYPES ================')
        print('---------------- COLLECTING VARIABLES ------------------')
    var_collector = VarCollector(context, errors)
    scope = var_collector.visit(ast)
    if debug:
        print('Errors: [')
        for error in errors:
            print('\t', error)
        print(']')
        print('Context:')
        print(context)
        print('Scope:')
        print(scope)
        print('---------------- INFERRING TYPES ------------------')
    type_inferrer = TypeInferer(context, errors)
    type_inferrer.visit(ast)

    # Check if there are any inference errors and change the types to ErrorType if there are
    # inference_errors = context.inference_errors() + scope.inference_errors()
    # errors.extend(inference_errors)
    if debug:
        # print('Iterations: ' + str(type_inferrer.current_iteration))
        print('Errors: [')
        for error in errors:
            print('\t', error)
        print(']')
        print('Context:')
        print(context)
        print('Scope:')
        print(scope)
        print('---------------- CHECKING TYPES ------------------')
    # type_checker = TypeChecker(context, errors)
    # type_checker.visit(ast)
    # if debug:
    #     print('Errors: [')
    #     for error in errors:
    #         print('\t', error)
    #     print(']')
        # print('Context:')
        # print(context)
        # print('Scope:')
        # print(scope)
    return ast, errors, context, scope


In [7]:
print(semantic_analysis_pipeline(asts[6],debug=True))

============== COLLECTING TYPES ===============
Errors: [
]
Context:
{
	type Unknow {}
	
	type Object {
		[method] equals(other:Object): Boolean;
		[method] toString(): String;
	}
	
	type Number : Object {}
	
	type Boolean : Object {}
	
	type String : Object {
		[method] size(): Number;
		[method] next(): Boolean;
		[method] current(): String;
	}
	
	type Range : Object {
		[attrib] min : Number;
		[attrib] max : Number;
		[attrib] current : Number;
		[method] next(): Boolean;
		[method] current(): Number;
	}
	
	type Point {}
	
	type PolarPoint {}
	
}
=============== BUILDING TYPES ================
visitando el tipo Point
visitando el tipo PolarPoint
Errors: [
]
Context:
{
	type Unknow {}
	
	type Object {
		[method] equals(other:Object): Boolean;
		[method] toString(): String;
	}
	
	type Number : Object {}
	
	type Boolean : Object {}
	
	type String : Object {
		[method] size(): Number;
		[method] next(): Boolean;
		[method] current(): String;
	}
	
	type Range : Object {
		[attrib] min :

AttributeError: 'str' object has no attribute 'name'

In [ ]:
print(semantic_analysis_pipeline(asts[7],debug=True))